In [ ]:
!pip install elasticsearch

In [16]:
import pandas as pd

In [17]:
from datetime import datetime
from elasticsearch import Elasticsearch


In [18]:
df = pd.DataFrame({'고객 세그먼트': ['EEE'],
                 '가치제안': ['EEE'],
                 '마케팅 채널': ['EEE'],
                 '고객관계': ['EEE'],
                 '수익원': ['EEE'],
                 '핵심활동': ['EEE'],
                 '핵심자원': ['EEE'],
                 '핵심파트너': ['EEE'],
                 '비용구조': ['EEE']})
df

,고객 세그먼트,가치제안,마케팅 채널,고객관계,수익원,핵심활동,핵심자원,핵심파트너,비용구조
0,EEE,EEE,EEE,EEE,EEE,EEE,EEE,EEE,EEE


In [19]:
df.iloc[0][3]

'EEE'

In [20]:
# elasticsearch 연결
es = Elasticsearch('http://34.64.84.121:9200')
es.info()

C:\Users\JuNoe\anaconda3\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'node-1',
 'cluster_name': 'es-cluster',
 'cluster_uuid': 'tYWuY5FGSRC51YMjqeyJeA',
 'version': {'number': '7.14.2',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '6bc13727ce758c0e943c3c21653b3da82f627f75',
  'build_date': '2021-09-15T10:18:09.722761972Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [27]:
# 동일한 이른의 인덱스가 있을 경우 삭제하고 다시 생성
def make_index(es, index_name):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        es.indices.create(index=index_name)

# 데이터베이스 이름 선언 (엘라스틱에서는 인덱스라 함)
index_name = 'bmc'

# elasticsearch에 텍스트 데이터프레임을 적재하는 함수
# df : 텍스트 9개 컬럼 데이터프레임
def input_data(df, index_name):

    # 데이터준비
    doc = { 
            '고객 세그먼트': df.iloc[0][0], 
            '가치제안': df.iloc[0][1], 
            "마케팅 채널": df.iloc[0][2],
            '고객관계': df.iloc[0][3], 
            "수익원": df.iloc[0][4],
            "핵심활동": df.iloc[0][5], 
            "핵심자원": df.iloc[0][6],
            "핵심파트너": df.iloc[0][7],
            "비용구조": df.iloc[0][8]}
    make_index(es, index_name)

    # 값 입력
    es.index(index=index_name, doc_type='string', body=doc)
    
    # 새로고침
    es.indices.refresh(index=index_name)

In [28]:
input_data(df, index_name)

In [29]:
results = es.search(index=index_name, body={'from':0, 'size':100, 'query':{'match':{'고객 세그먼트':'EEE'}}})
for result in results['hits']['hits']:
    print('score:', result['_score'], 'source:', result['_source'])

score: 0.2876821 source: {'고객 세그먼트': 'EEE', '가치제안': 'EEE', '마케팅 채널': 'EEE', '고객관계': 'EEE', '수익원': 'EEE', '핵심활동': 'EEE', '핵심자원': 'EEE', '핵심파트너': 'EEE', '비용구조': 'EEE'}
